# Quick start

- To check data and model if it preditcs or not.
- This is only for intent dataset
- Model implemented in for 8640 data samples after filtering

# 1)- Import Key Modules

In [1]:
# support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pickle
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [3]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

In [4]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


# 2)- Loading Data

In [5]:
data=pd.read_pickle('file_clean_intent.pkl')

In [6]:
data.dep.nunique()

30

In [7]:
counts=data['dep'].value_counts()
df = data.loc[data['dep'].isin(counts.index[counts > 150])]

In [8]:
print(data.shape)
print(df.shape)

(9477, 4)
(8640, 4)


In [9]:
df.dep.nunique()

10

In [10]:
myfeat=["Order management","product complaints - products (Reklamation Produkte)","Software/Webshop/App",
        "Shipping issues","Customer feedback","ShareWithSaal","Payment (Bezahlung)",
        "product complaints - colours (Reklamation Farben)","Product (Produkt)","Marketing"]

### Data Clean

In [11]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['clean'] = df['firstmessage'].apply(clean_text)

### Train-Test

In [12]:
train_size = int(len(df) * 0.70)
train_posts = df['clean'][:train_size]
train_tags = df['dep'][:train_size]

test_posts = df['clean'][train_size:]
test_tags = df['dep'][train_size:]

In [13]:
print(train_posts.shape)

(6048,)


### Tokenization

In [14]:
max_words = 10000 #original dim is 35250
tokenize = text.Tokenizer(num_words=max_words,char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [15]:
print(x_train.shape)

(6048, 10000)


In [16]:
x_train[:5]

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### encoder

only for target i.e label data

In [17]:
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags) 
y_test = encoder.transform(test_tags)

In [18]:
num_classes = np.max(y_train) + 1 #strings start from 0 index. To count them from 1-10
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [19]:
print(y_train.shape)
print(y_test.shape)

(6048, 10)
(2592, 10)


### Model Building

In [20]:
batch_size = 32
epochs = 10
shape_of_input=max_words # 1000 words or X_pd_sq.shape[1]

In [21]:
model = Sequential()
model.add(Dense(512, input_shape=(shape_of_input,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))#The dropout rate is set to 20%, meaning one in 5 inputs will be randomly excluded from each update cycle
model.add(Dense(num_classes)) # there are 10 classes i.e at output layer
model.add(Activation('softmax'))

In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               5120512   
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 5,125,642
Trainable params: 5,125,642
Non-trainable params: 0
_________________________________________________________________


In [24]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 5443 samples, validate on 605 samples
Epoch 1/10
5443/5443 [==============================] - 12s 2ms/step - loss: 1.4410 - accuracy: 0.5594 - val_loss: 1.2957 - val_accuracy: 0.6066
Epoch 2/10
5443/5443 [==============================] - 12s 2ms/step - loss: 0.7907 - accuracy: 0.7523 - val_loss: 1.2877 - val_accuracy: 0.6413
Epoch 3/10
5443/5443 [==============================] - 12s 2ms/step - loss: 0.4933 - accuracy: 0.8558 - val_loss: 1.4027 - val_accuracy: 0.6397
Epoch 4/10
5443/5443 [==============================] - 14s 3ms/step - loss: 0.3721 - accuracy: 0.8967 - val_loss: 1.5280 - val_accuracy: 0.6331
Epoch 5/10
5443/5443 [==============================] - 13s 2ms/step - loss: 0.3326 - accuracy: 0.9074 - val_loss: 1.7398 - val_accuracy: 0.6380
Epoch 6/10
5443/5443 [==============================] - 12s 2ms/step - loss: 0.3032 - accuracy: 0.9148 - val_loss: 1.7896 - val_accuracy: 0.6264
Epoch 7/10
5443/5443 [==============================] - 12s 2ms/step - loss: 0.2713

We can see overfitting pattern is our model

In [25]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

2592/2592 [==============================] - 1s 401us/step
Test accuracy: 0.6280864477157593


# END OF NOTEBOOK